## Load required packages

In [2]:
from autogen import ConversableAgent, UserProxyAgent

In [3]:

import os
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()


True

## Configure LLM

In [4]:
# use gpt-4o model, as gemini is not working
llm_config = {
    "model": "gpt-4o",
    "api_key": os.getenv("OPENAI_API_KEY")
    }

# Create Research Agents

## Manager Agent

In [5]:
manager_proxy_agent = UserProxyAgent(
    name="Manager",
    system_message="You are a marketing campaign manager who is responsible for providing campaign ideas and evaluate them and scripts generation.",
    human_input_mode="NEVER",
    llm_config=llm_config,
    code_execution_config=False
)


## Idea generation agent

In [7]:
idea_generator = ConversableAgent(
    name="IdeaGenerator",
    system_message="""
    You are a creative assistant that helps generate marketing campaign ideas.
    1. generate innovative campaign ideas based on the given topic,
    2. suggest a potential storyline for the campaign and chracters for video content,
    3. provide a brief outline for each idea.
    Present 3-5 unique ideas for each request.
    """,
    human_input_mode="NEVER",
    llm_config=llm_config
)


## Script writing agent

In [8]:
script_writer = ConversableAgent(
    name="ScriptWriter",
    system_message="""    You are a script writer that helps generate marketing video scripts.
    1. develope full scripts based on the provided campaign idea,
    2. include dialogues, scene descriptions, and any necessary details for video production.
    3. ensure the script aligns with the campaign's objectives and target audience.
    Aim for scripts of 2-3 minutes in length.
    """,
    human_input_mode="NEVER",
    llm_config=llm_config
)

## Script review agent

In [9]:
script_reviewer = ConversableAgent(
    name="ScriptReviewer",
    system_message="""    You are a script reviewer that helps review marketing video scripts.
    1. evaluate the provided script for clarity, engagement, and alignment with the campaign's objectives,
    2. provide constructive feedback and suggestions for improvement,
    3. ensure the script is suitable for the target audience and platform.
    Be thorough and specific in your feedback.
    """,
    human_input_mode="NEVER",
    llm_config=llm_config
)

# Create tasks for agents

In [12]:
# a sequenctial chat between agents
chats = [
    {
        "sender": manager_proxy_agent,
        "recipient": idea_generator,
        "message": "Generate innovative marketing campaign ideas for a new eco-friendly product launch. Focus on sustainability and community engagement.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Summarize the generated ideas into a JSON object:"
            "{idea1: {'concept': '', 'storyline': '', 'characters': ''},"
            "idea2: {...}, 'idea3': {...} }"
        },
        "max_turns": 2,
        "clear_history": True
    },
    {
        "sender": manager_proxy_agent,
        "recipient": script_writer,
        "message": "Write a script for the first idea generated by the IdeaGenerator. The script should be engaging and suitable for a 2-3 minute video.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Summarize the script into a JSON object:"
            "{'title': '', 'main_characters': [], 'key_scenes': [], 'estimated_duration': ''}"
        },
        "max_turns": 2,
        "clear_history": False
    },
    {
        "sender": manager_proxy_agent,
        "recipient": script_reviewer,
        "message": "Review the script generated by the ScriptWriter. Ensure it aligns with the campaign's objectives and is suitable for the target audience. Provide constructive feedback and suggestions for improvement.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Summarize the review feedback into a JSON object:"
            "{'strengths': '', 'weaknesses': [], 'suggestions': [], 'overall_rating': ''}"
        },
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": manager_proxy_agent,
        "recipient": manager_proxy_agent,
        "message": "Based on the script and review feedback, decide whether to proceed with the script as is, make revisions, or generate a new script.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Summarize the decision into a JSON object:"
            "{'decision': '', 'next_steps': []}"
        },
        "max_turns": 1,
        "clear_history": False
    }
]

## Start researching

In [14]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Manager (to IdeaGenerator):

Generate innovative marketing campaign ideas for a new eco-friendly product launch. Focus on sustainability and community engagement.

--------------------------------------------------------------------------------
IdeaGenerator (to Manager):

1. **Campaign Name: "Green Footprints"**

   - **Storyline:** The campaign follows the journey of a single seed as it moves through different community hands, each adding their unique touch and care. The seed grows into a beautiful tree, symbolizing the cumulative impact of small, sustainable actions. Characters include an elderly woman who teaches the young about composting, a schoolteacher who involves her students in planting, and a local business owner converting trash into plant pots.
   
   - **Outline:**
     1. Launch an int

In [15]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
    "idea1": {
        "concept": "Green Footprints",
        "storyline": "This campaign follows the journey of a seed as it travels through different community hands, symbolizing the cumulative impact of small, sustainable actions.",
        "characters": "An elderly woman, a schoolteacher, community members."
    },
    "idea2": {
        "concept": "Eco Innovators",
        "storyline": "A group of diverse characters comes together to solve community environmental issues using the new product, showcasing creativity and innovation.",
        "characters": "A teenage inventor, an artist, a retired engineer."
    },
    "idea3": {
        "concept": "Sustainable Hearts Club",
        "storyline": "A love story unfolds between two young environmentalists who meet at a beach cleanup, emphasizing personal connections through shared sustainable goals.",
        "characters": "Two young environmentalists."
    },
    "idea4": {
        "concept": "Change Makers Chronicles",
     

In [16]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.036930000000000004, 'gpt-4o-2024-08-06': {'cost': 0.036930000000000004, 'prompt_tokens': 4892, 'completion_tokens': 2470, 'total_tokens': 7362}}, 'usage_excluding_cached_inference': {'total_cost': 0.036930000000000004, 'gpt-4o-2024-08-06': {'cost': 0.036930000000000004, 'prompt_tokens': 4892, 'completion_tokens': 2470, 'total_tokens': 7362}}}


{'usage_including_cached_inference': {'total_cost': 0.0524625, 'gpt-4o-2024-08-06': {'cost': 0.0524625, 'prompt_tokens': 7597, 'completion_tokens': 3347, 'total_tokens': 10944}}, 'usage_excluding_cached_inference': {'total_cost': 0.0524625, 'gpt-4o-2024-08-06': {'cost': 0.0524625, 'prompt_tokens': 7597, 'completion_tokens': 3347, 'total_tokens': 10944}}}


{'usage_including_cached_inference': {'total_cost': 0.033562499999999995, 'gpt-4o-2024-08-06': {'cost': 0.033562499999999995, 'prompt_tokens': 3877, 'completion_tokens': 2387, 'total_tokens': 6264}}, 'usage_excluding_cached_inference': {'to